In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
map_df = gpd.read_file('../src_data/us_county_mainland/us_county.shp')

In [3]:
case_df = pd.read_csv('covid19-us-counties.csv', dtype={'fips':str})

In [4]:
case_df.loc[case_df[case_df['county'] == 'New York City'].index, 'fips'] = '36061'
case_df.loc[case_df[case_df['county'] == 'Kansas City'].index, 'fips'] = '29095'
case_df.loc[case_df[case_df['county'] == 'Joplin'].index, 'fips'] =  '29093'
case_df = case_df[case_df['county'] != 'Unkown']
case_df['month'] = case_df['date'].apply(lambda x: x[:7])
case_df

,date,county,state,fips,cases,deaths,month
0,2020-01-21,Snohomish,Washington,53061,1,0.0,2020-01
1,2020-01-22,Snohomish,Washington,53061,1,0.0,2020-01
2,2020-01-23,Snohomish,Washington,53061,1,0.0,2020-01
3,2020-01-24,Cook,Illinois,17031,1,0.0,2020-01
4,2020-01-24,Snohomish,Washington,53061,1,0.0,2020-01
...,...,...,...,...,...,...,...
2502827,2022-05-13,Sweetwater,Wyoming,56037,11088,126.0,2022-05
2502828,2022-05-13,Teton,Wyoming,56039,10074,16.0,2022-05
2502829,2022-05-13,Uinta,Wyoming,56041,5643,39.0,2022-05
2502830,2022-05-13,Washakie,Wyoming,56043,2358,44.0,2022-05


In [5]:
clean_df = case_df.groupby(['month','fips']).tail(1).reset_index()
clean_df = clean_df[['month', 'fips', 'cases']].reset_index()
clean_df

,index,month,fips,cases
0,0,2020-01,04013,1
1,1,2020-01,06037,1
2,2,2020-01,06059,1
3,3,2020-01,06085,1
4,4,2020-01,17031,2
...,...,...,...,...
85130,85130,2022-05,56037,11088
85131,85131,2022-05,56039,10074
85132,85132,2022-05,56041,5643
85133,85133,2022-05,56043,2358


In [6]:
pop_df = pd.read_csv('../src_data/usa_county_population.csv', dtype={'GeoId':str})
pop_df

,GeoId,Population
0,00003,1000
1,01001,55380
2,01003,213830
3,01005,25361
4,01007,22493
...,...,...
3216,72145,53192
3217,72147,9642
3218,72149,22403
3219,72151,34499


In [7]:
clean_df = clean_df.join(pop_df.set_index('GeoId'), on='fips', how = 'left')
clean_df['Percent-Infected'] = clean_df['cases']/clean_df['Population']
clean_df

,index,month,fips,cases,Population,Percent-Infected
0,0,2020-01,04013,1,4333810.0,2.307438e-07
1,1,2020-01,06037,1,10110570.0,9.890639e-08
2,2,2020-01,06059,1,3170044.0,3.154530e-07
3,3,2020-01,06085,1,1928470.0,5.185458e-07
4,4,2020-01,17031,2,5205275.0,3.842256e-07
...,...,...,...,...,...,...
85130,85130,2022-05,56037,11088,43521.0,2.547736e-01
85131,85131,2022-05,56039,10074,23280.0,4.327320e-01
85132,85132,2022-05,56041,5643,20479.0,2.755506e-01
85133,85133,2022-05,56043,2358,8027.0,2.937586e-01


In [8]:
months = clean_df['month'].unique().tolist()

In [18]:
for month in months:
    fig, ax = plt.subplots(figsize = (20,20))
    
    plot_data = clean_df[clean_df['month'] == month].reset_index(drop=True)
    percentiles_I = plot_data[plot_data['cases'] != 0]['cases'].quantile(0.95)
    prob_map_df = map_df.merge(plot_data, left_on='GEOID', right_on='fips', how = 'left').fillna(0)
    prob_map_df.plot(column='cases', cmap='Reds',  edgecolor='#255e30', ax = ax, linewidth = 0.1, legend=True , vmin=0, vmax = percentiles_I, legend_kwds={'shrink': 0.25})
    ax.axis('off')
    ax.set_title(f'{month}')
    fig_fn = f'figures/{month}.png'
    fig.savefig(fig_fn, bbox_inches="tight")
    plt.close(fig)

In [9]:
from PIL import Image, ImageOps
import numpy as np

gif_fn = 'real_case.gif'
images_ori = [Image.open(f'figures/{month}.png') for month in months]
target_size = images_ori[-1].size
images = [ImageOps.pad(img, target_size, color ='white') for img in images_ori]

images[0].save(
    gif_fn,
    save_all=True,
    append_images=images[1:],
    duration=240,
    loop=0
)

In [19]:
from PIL import Image, ImageOps
import cv2
import numpy as np
import os

# List of image paths
image_paths = [f'figures/{month}.png' for month in months]  # replace with your paths

# Load first image to get size
first_frame = Image.open(image_paths[-1])
frame_array = np.array(first_frame)
height, width, _ = frame_array.shape

# Define output video writer
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 12, (width, height))

for img_path in image_paths:
    img = Image.open(img_path).convert('RGB')
    img = ImageOps.pad(img, first_frame.size, color ='white')
    frame = np.array(img)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # convert RGB to BGR for OpenCV
    out.write(frame)

out.release()
print("Video saved as output_video.mp4")


Video saved as output_video.mp4
